In [ ]:
import os
import json
import openslide
from PIL import Image
import cv2
import numpy as np
from tqdm import tqdm
from glob import glob


In [ ]:
json_list=glob('../../data/IHC_HE_Pair_Data_GA_hospital/registration_params/*.json')
i=1
with open(json_list[i],'r') as f:
    json_data=json.load(f)

he_slide=openslide.OpenSlide(json_data['files']['he_slide'])
pdl1_slide=openslide.OpenSlide(json_data['files']['pdl1_slide'])
json_data

In [ ]:
# 썸네일 및 마스크 생성 함수
def get_thumbnail_and_mask(slide_path, downsample=30):
    """슬라이드에서 썸네일과 조직 마스크 생성"""
    slide = openslide.OpenSlide(slide_path)
    thumb = slide.get_thumbnail((slide.dimensions[0]//downsample, slide.dimensions[1]//downsample))
    thumb_np = np.array(thumb)
    
    # Grayscale 변환 및 Otsu threshold
    gray = cv2.cvtColor(thumb_np, cv2.COLOR_RGB2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # 모폴로지 연산으로 노이즈 제거
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    return slide, thumb_np, mask

# HE와 PD-L1 썸네일 및 마스크 생성
he_slide, he_thumb_np, he_mask = get_thumbnail_and_mask(json_data['files']['he_slide'])
pdl1_slide, pdl1_thumb_np, pdl1_mask = get_thumbnail_and_mask(json_data['files']['pdl1_slide'])

print(f"HE thumbnail shape: {he_thumb_np.shape}, mask tissue ratio: {np.sum(he_mask > 0) / he_mask.size:.2%}")
print(f"PD-L1 thumbnail shape: {pdl1_thumb_np.shape}, mask tissue ratio: {np.sum(pdl1_mask > 0) / pdl1_mask.size:.2%}")

In [ ]:
# HE 썸네일에 transformation 적용
from scipy.ndimage import rotate

def apply_transformation_to_thumbnail(thumb_np, mask, params):
    """썸네일과 마스크에 transformation 적용"""
    angle = params['transformation']['thumbnail']['angle_degrees']
    scale = params['transformation']['thumbnail']['scale']
    tx = int(params['transformation']['thumbnail']['translation_x'])
    ty = int(params['transformation']['thumbnail']['translation_y'])
    
    # 이미지 회전
    rotated_img = rotate(thumb_np, angle, reshape=False, order=1)
    rotated_mask = rotate(mask, angle, reshape=False, order=0)
    
    # 이미지 스케일링
    new_h, new_w = int(thumb_np.shape[0] * scale), int(thumb_np.shape[1] * scale)
    scaled_img = cv2.resize(rotated_img, (new_w, new_h))
    scaled_mask = cv2.resize(rotated_mask.astype(np.uint8), (new_w, new_h), interpolation=cv2.INTER_NEAREST)
    
    # PD-L1 크기에 맞춰 결과 이미지 생성
    target_h, target_w = params['dimensions']['pdl1_thumbnail']['height'], params['dimensions']['pdl1_thumbnail']['width']
    result_img = np.zeros((target_h, target_w, 3), dtype=np.uint8)
    result_mask = np.zeros((target_h, target_w), dtype=np.uint8)
    
    # 이동(translation) 적용
    start_y = (target_h - scaled_img.shape[0]) // 2 + ty
    start_x = (target_w - scaled_img.shape[1]) // 2 + tx
    
    src_start_y = max(0, -start_y)
    src_start_x = max(0, -start_x)
    src_end_y = min(scaled_img.shape[0], target_h - start_y)
    src_end_x = min(scaled_img.shape[1], target_w - start_x)
    
    dst_start_y = max(0, start_y)
    dst_start_x = max(0, start_x)
    dst_end_y = dst_start_y + (src_end_y - src_start_y)
    dst_end_x = dst_start_x + (src_end_x - src_start_x)
    
    if src_end_y > src_start_y and src_end_x > src_start_x:
        result_img[dst_start_y:dst_end_y, dst_start_x:dst_end_x] = scaled_img[src_start_y:src_end_y, src_start_x:src_end_x]
        result_mask[dst_start_y:dst_end_y, dst_start_x:dst_end_x] = scaled_mask[src_start_y:src_end_y, src_start_x:src_end_x]
    
    return result_img, result_mask

# HE 썸네일을 PD-L1 공간으로 변환
he_transformed_thumb, he_transformed_mask = apply_transformation_to_thumbnail(he_thumb_np, he_mask, json_data)

print(f"HE transformed shape: {he_transformed_thumb.shape}")
print(f"HE transformed mask tissue ratio: {np.sum(he_transformed_mask > 0) / he_transformed_mask.size:.2%}")

In [ ]:
# 공통 영역 추출
common_mask = np.logical_and(he_transformed_mask > 0, pdl1_mask > 0).astype(np.uint8) * 255

print(f"Common area ratio: {np.sum(common_mask > 0) / common_mask.size:.2%}")

# 시각화
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1행: 마스크
axes[0, 0].imshow(he_transformed_mask, cmap='gray')
axes[0, 0].set_title('HE Mask (Transformed)')
axes[0, 0].axis('off')

axes[0, 1].imshow(pdl1_mask, cmap='gray')
axes[0, 1].set_title('PD-L1 Mask')
axes[0, 1].axis('off')

axes[0, 2].imshow(common_mask, cmap='gray')
axes[0, 2].set_title('Common Area Mask')
axes[0, 2].axis('off')

# 2행: 오버레이
# 마스크 오버레이 (Red=HE, Blue=PD-L1, Purple=Both)
mask_overlay = np.zeros((*pdl1_mask.shape, 3), dtype=np.uint8)
mask_overlay[he_transformed_mask > 0] = [255, 0, 0]
mask_overlay[pdl1_mask > 0] = mask_overlay[pdl1_mask > 0] + [0, 0, 255]

axes[1, 0].imshow(mask_overlay)
axes[1, 0].set_title('Mask Overlay (Red=HE, Blue=PD-L1, Purple=Both)')
axes[1, 0].axis('off')

# 이미지 오버레이
alpha = 0.5
img_overlay = cv2.addWeighted(he_transformed_thumb, alpha, pdl1_thumb_np, 1-alpha, 0)
axes[1, 1].imshow(img_overlay)
axes[1, 1].set_title('Image Overlay (HE+PD-L1)')
axes[1, 1].axis('off')

# 공통영역만 표시
common_area_img = pdl1_thumb_np.copy()
common_area_img[common_mask == 0] = [255, 255, 255]  # 공통영역이 아닌 곳은 흰색
axes[1, 2].imshow(common_area_img)
axes[1, 2].set_title('Common Area Only')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def transform_point_pdl1_to_he(pdl1_x, pdl1_y, params):
    """
    PD-L1 WSI 좌표의 한 점을 HE WSI 좌표로 역변환
    Forward (HE → PD-L1): (중심이동) → rotate → scale → (중앙배치 + translate)
    Inverse (PD-L1 → HE): (중앙배치 + translate)⁻¹ → scale⁻¹ → rotate⁻¹ → (중심복원)
    """
    # Full resolution 변환 파라미터 사용
    angle = params['transformation']['fullres']['angle_degrees']
    scale = params['transformation']['fullres']['scale']
    tx = params['transformation']['fullres']['translation_x']
    ty = params['transformation']['fullres']['translation_y']
    
    # HE WSI 크기
    he_width = params['dimensions']['he_full']['width']
    he_height = params['dimensions']['he_full']['height']
    he_center_x = he_width / 2
    he_center_y = he_height / 2
    
    # PD-L1 WSI 크기
    pdl1_width = params['dimensions']['pdl1_full']['width']
    pdl1_height = params['dimensions']['pdl1_full']['height']
    pdl1_center_x = pdl1_width / 2
    pdl1_center_y = pdl1_height / 2

    # 1. Translation 역변환 (PD-L1 중심 기준으로 변환)
    # Forward에서: scaled 이미지가 PD-L1 중앙에 배치 + (tx, ty) 이동
    # Inverse: PD-L1 절대좌표 → PD-L1 중심기준 → translation 제거
    x = pdl1_x - pdl1_center_x - tx
    y = pdl1_y - pdl1_center_y - ty

    # 2. Scale 역변환
    x = x / scale
    y = y / scale
    
    # 3. Rotation 역변환 (원점 기준)
    angle_rad = np.radians(angle)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    
    rotated_x = x * cos_a - y * sin_a
    rotated_y = x * sin_a + y * cos_a
    
    # 4. HE 중심 복원 (HE 절대좌표로)
    x = rotated_x + he_center_x
    y = rotated_y + he_center_y
    
    return x, y


def transform_rectangle_pdl1_to_he(pdl1_corners, params):
    """
    PD-L1 WSI 좌표의 사각형 4개 코너를 HE WSI 좌표로 변환
    pdl1_corners: numpy array of shape (4, 2) - [[x1,y1], [x2,y2], [x3,y3], [x4,y4]]
    """
    he_corners = []
    for corner in pdl1_corners:
        he_x, he_y = transform_point_pdl1_to_he(corner[0], corner[1], params)
        he_corners.append([he_x, he_y])
    return np.array(he_corners)


def get_patch_corners(center_x, center_y, patch_size):
    """
    중점을 기준으로 패치의 4개 코너 좌표 반환 (좌상단부터 시계방향)
    """
    half_size = patch_size / 2
    corners = np.array([
        [center_x - half_size, center_y - half_size],  # 좌상단
        [center_x + half_size, center_y - half_size],  # 우상단
        [center_x + half_size, center_y + half_size],  # 우하단
        [center_x - half_size, center_y + half_size]   # 좌하단
    ])
    return corners


# mpp 1.0 기준 1024x1024 패치 크기 (픽셀 단위)
# json_data에서 mpp 정보 가져오기
pdl1_mpp = json_data.get('mpp', {}).get('pdl1', 0.5)  # 기본값 0.5
he_mpp = json_data.get('mpp', {}).get('he', 0.5)  # 기본값 0.5

# mpp 1.0 기준으로 변환
target_mpp = 2.0
pdl1_patch_size_wsi = int(1024 * (target_mpp / pdl1_mpp))  # mpp 1.0에서 1024 -> 현재 mpp로 변환
he_patch_size_wsi = int(1024 * (target_mpp / he_mpp))

# 썸네일 스케일 계산
pdl1_thumb_scale_x = pdl1_thumb_np.shape[1] / pdl1_slide.dimensions[0]
pdl1_thumb_scale_y = pdl1_thumb_np.shape[0] / pdl1_slide.dimensions[1]
he_thumb_scale_x = he_thumb_np.shape[1] / he_slide.dimensions[0]
he_thumb_scale_y = he_thumb_np.shape[0] / he_slide.dimensions[1]

# WSI 레벨에서 테스트 포인트 설정 (PD-L1 full resolution 좌표)
# common_mask 내부에서 랜덤하게 샘플링
num_test_points = 5

# 썸네일에서 유효한 좌표 찾기 (common_mask > 0인 영역)
valid_thumb_coords = np.argwhere(common_mask > 0)

# 랜덤하게 num_test_points개 선택
np.random.seed(42)  # 재현성을 위한 시드
selected_indices = np.random.choice(len(valid_thumb_coords), size=num_test_points, replace=False)
selected_thumb_coords = valid_thumb_coords[selected_indices]

# 썸네일 좌표를 WSI 좌표로 변환
test_points_wsi = []
for thumb_y, thumb_x in selected_thumb_coords:
    wsi_x = int(thumb_x / pdl1_thumb_scale_x)
    wsi_y = int(thumb_y / pdl1_thumb_scale_y)
    test_points_wsi.append((wsi_x, wsi_y))

print(f"랜덤 샘플링된 {len(test_points_wsi)}개의 테스트 포인트:")
for idx, (x, y) in enumerate(test_points_wsi):
    print(f"  Point {idx+1}: ({x}, {y})")

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# PD-L1 썸네일 + 포인트 및 패치
pdl1_vis = pdl1_thumb_np.copy()
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]

print(f"mpp 정보: PD-L1={pdl1_mpp}, HE={he_mpp}")
print(f"mpp {target_mpp} 기준 1024x1024 패치:")
print(f"  PD-L1 WSI에서: {pdl1_patch_size_wsi}x{pdl1_patch_size_wsi} 픽셀")
print(f"  HE WSI에서: {he_patch_size_wsi}x{he_patch_size_wsi} 픽셀")
print(f"\nPD-L1 WSI 크기: {pdl1_slide.dimensions}")
print(f"HE WSI 크기: {he_slide.dimensions}")
print(f"\nPD-L1 썸네일 스케일: x={pdl1_thumb_scale_x:.6f}, y={pdl1_thumb_scale_y:.6f}")
print(f"HE 썸네일 스케일: x={he_thumb_scale_x:.6f}, y={he_thumb_scale_y:.6f}")
print("\n좌표 및 패치 변환 결과:")

for idx, (px, py) in enumerate(test_points_wsi):
    # PD-L1 패치의 4개 코너 계산
    pdl1_corners = get_patch_corners(px, py, pdl1_patch_size_wsi)
    
    # PD-L1 WSI 좌표를 썸네일 좌표로 변환하여 시각화
    pdl1_thumb_center_x = int(px * pdl1_thumb_scale_x)
    pdl1_thumb_center_y = int(py * pdl1_thumb_scale_y)
    
    pdl1_thumb_corners = pdl1_corners * [pdl1_thumb_scale_x, pdl1_thumb_scale_y]
    pdl1_thumb_corners = pdl1_thumb_corners.astype(np.int32)
    
    color = colors[idx % len(colors)]
    
    # 패치 영역 다각형 그리기
    cv2.polylines(pdl1_vis, [pdl1_thumb_corners], True, color, 2)
    
    # 중점 표시
    cv2.circle(pdl1_vis, (pdl1_thumb_center_x, pdl1_thumb_center_y), 8, color, -1)
    cv2.circle(pdl1_vis, (pdl1_thumb_center_x, pdl1_thumb_center_y), 10, (0, 0, 0), 2)
    
    # 번호 표시
    cv2.putText(pdl1_vis, f'{idx+1}', (pdl1_thumb_center_x - 10, pdl1_thumb_center_y - 15), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

axes[0].imshow(pdl1_vis)
axes[0].set_title(f'PD-L1 Slide (mpp {target_mpp}: {pdl1_patch_size_wsi}x{pdl1_patch_size_wsi} pixels)\n(Visualization on Thumbnail)', 
                  fontsize=14, fontweight='bold')
axes[0].axis('off')

# HE 썸네일 + 변환된 중점과 패치 영역
he_vis = he_thumb_np.copy()

for idx, (px, py) in enumerate(test_points_wsi):
    # PD-L1 패치의 4개 코너 계산
    pdl1_corners = get_patch_corners(px, py, pdl1_patch_size_wsi)
    
    # PD-L1 코너들을 HE WSI 좌표로 변환
    he_corners = transform_rectangle_pdl1_to_he(pdl1_corners, json_data)
    
    # 중점도 변환
    he_wsi_x, he_wsi_y = transform_point_pdl1_to_he(px, py, json_data)
    
    # HE WSI 좌표를 썸네일 좌표로 변환하여 시각화
    he_thumb_center_x = int(he_wsi_x * he_thumb_scale_x)
    he_thumb_center_y = int(he_wsi_y * he_thumb_scale_y)
    
    he_thumb_corners = he_corners * [he_thumb_scale_x, he_thumb_scale_y]
    he_thumb_corners = he_thumb_corners.astype(np.int32)
    
    color = colors[idx % len(colors)]
    
    # 변환된 패치 영역 다각형 그리기
    cv2.polylines(he_vis, [he_thumb_corners], True, color, 2)
    
    # 중점 표시 (경계 체크)
    if (0 <= he_thumb_center_x < he_thumb_np.shape[1] and 
        0 <= he_thumb_center_y < he_thumb_np.shape[0]):
        cv2.circle(he_vis, (he_thumb_center_x, he_thumb_center_y), 8, color, -1)
        cv2.circle(he_vis, (he_thumb_center_x, he_thumb_center_y), 10, (0, 0, 0), 2)
        
        # 번호 표시
        cv2.putText(he_vis, f'{idx+1}', (he_thumb_center_x - 10, he_thumb_center_y - 15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    
    print(f"  Point {idx+1}:")
    print(f"    PD-L1 WSI center: ({px:.1f}, {py:.1f})")
    print(f"    HE WSI center:    ({he_wsi_x:.1f}, {he_wsi_y:.1f})")
    print(f"    PD-L1 corners (WSI): {pdl1_corners[0]} -> {pdl1_corners[2]}")
    print(f"    HE corners (WSI):    {he_corners[0]} -> {he_corners[2]}")

axes[1].imshow(he_vis)
axes[1].set_title(f'HE Slide (mpp {target_mpp}: {he_patch_size_wsi}x{he_patch_size_wsi} pixels)\n(Transformed Patches, Visualization on Thumbnail)', 
                  fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# 패치 추출 함수
def extract_patch(slide, center_x, center_y, patch_size,patch_resize=1024, level=0):
    """
    WSI에서 중점 기준으로 패치 추출
    
    Args:
        slide: OpenSlide 객체
        center_x, center_y: 패치 중점 좌표 (level 0 기준)
        patch_size: 패치 크기 (level 0 기준)
        level: 추출할 레벨
    
    Returns:
        patch: numpy array (RGB)
    """
    # 좌상단 좌표 계산
    x = int(center_x - patch_size / 2)
    y = int(center_y - patch_size / 2)
    
    # 경계 체크
    x = max(0, min(x, slide.dimensions[0] - patch_size))
    y = max(0, min(y, slide.dimensions[1] - patch_size))
    
    # 패치 읽기
    patch = slide.read_region((x, y), level, (patch_size, patch_size))
    patch = np.array(patch.convert('RGB'))
    
    
    # 패치 크기 조정 (resize)
    if patch_resize != patch_size:
        patch = cv2.resize(patch, (patch_resize, patch_resize), interpolation=cv2.INTER_LINEAR)
    return patch


# 테스트: 5개 포인트에서 패치 추출 및 시각화
print("패치 추출 중...")
print(f"PD-L1 패치 크기: {pdl1_patch_size_wsi}x{pdl1_patch_size_wsi}")
print(f"HE 패치 크기: {he_patch_size_wsi}x{he_patch_size_wsi}")
print()

# 시각화 준비
num_points = len(test_points_wsi)
fig, axes = plt.subplots(num_points, 2, figsize=(12, 6*num_points))

if num_points == 1:
    axes = axes.reshape(1, -1)

for idx, (px, py) in enumerate(test_points_wsi):
    print(f"Point {idx+1}: PD-L1 ({px}, {py})")
    
    # 1. PD-L1 패치 추출
    pdl1_patch = extract_patch(pdl1_slide, px, py, pdl1_patch_size_wsi, level=0)
    
    # 2. PD-L1 좌표를 HE 좌표로 변환
    he_x, he_y = transform_point_pdl1_to_he(px, py, json_data)
    print(f"         HE ({he_x:.1f}, {he_y:.1f})")
    
    # 3. HE 패치 추출
    he_patch = extract_patch(he_slide, he_x, he_y, he_patch_size_wsi, level=0)
    
    # 4. 시각화
    axes[idx, 0].imshow(pdl1_patch)
    axes[idx, 0].set_title(f'Point {idx+1} - PD-L1 Patch\n({px}, {py})', 
                           fontsize=12, fontweight='bold')
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(he_patch)
    axes[idx, 1].set_title(f'Point {idx+1} - HE Patch\n({he_x:.0f}, {he_y:.0f})', 
                           fontsize=12, fontweight='bold')
    axes[idx, 1].axis('off')
    
    print(f"  PD-L1 patch shape: {pdl1_patch.shape}")
    print(f"  HE patch shape: {he_patch.shape}")
    print()

plt.tight_layout()
plt.show()

print("패치 추출 완료!")
